In [15]:
publish.html("""
Cloud Formation:
<select id='action'>
 <option value="none" selected>none</option>
 <option value="create" >create</option>
 <option value="remove">remove</option>
 <option value="suspend">suspend</option>
 <option value="resume">resume</option>
</select>
<button id="run">Run</button>
<br><div id='output'></div>
""")

Cloud Formation:
 
 none 
 create 
 remove 
 suspend 
 resume 
 
 Run

In [16]:
publish.javascript("""
  window.now = function() { 
     var time = new Date(); 
     return time.getHours()+":"+time.getMinutes()+":"+time.getSeconds() 
  }
  $("#run").off("click")
  $("#run").on("click", function(data) {
  var data = "val action=\""+$("#action").val()+"\"\nval debug=false"
  IPython.notebook.get_cell(2).code_mirror.setValue(data) 
  IPython.notebook.select(2)
  IPython.notebook.execute_cells_below()
})
""")
def message(s:String) = publish.javascript("$('#output').text(now()+' "+s+"')")


defined function message

In [24]:
val action="create"
val debug=false

action: String = "create"
debug: Boolean = false

In [25]:
// options
import ammonite.ops._

val stackName = "test"
/// parameters
// template is for us-east-1
// change images for different regions
val region = "us-east-1"
val imageId = "ami-10f53a06"
val instanceType = "t2.small"
val keyName = "atomic"
val template = pwd/'aws/"kubernetes.yml"

import ammonite.ops._


stackName: String = "test"
region: String = "us-east-1"
imageId: String = "ami-10f53a06"
instanceType: String = "t2.small"
keyName: String = "atomic"
template: Path = /Users/msciab/Dropbox/Work/MacBookPro/MicheleSciabarra/Mosaico/cloud/aws/kubernetes.yml

In [26]:
import $ivy.`com.amazonaws:aws-java-sdk:1.11.22`

import $ivy.$                                   

In [27]:
import com.amazonaws.services.cloudformation.model._
import collection.JavaConverters._

val create  = action  == "create"
val remove  = action  == "remove"
val suspend = action  == "suspend"
val resume  = action  == "resume"

import com.amazonaws.services.cloudformation.model._

import collection.JavaConverters._


create: Boolean = true
remove: Boolean = false
suspend: Boolean = false
resume: Boolean = false

In [28]:
import com.amazonaws.services._
import com.amazonaws.services.cloudformation._

val cf = new AmazonCloudFormationClient() 
def status = {
    val stack = cf.describeStacks
        .getStacks.asScala
        .filter( _.getStackName == stackName).headOption
    stack.map(_.getStackStatus)
}
def statusLoop {
    val st = status
    message(st.getOrElse("-"))
    if(st.nonEmpty && st.get.endsWith("_IN_PROGRESS")) {
        Thread.sleep(1000)
        statusLoop
    }
}

import com.amazonaws.services._

import com.amazonaws.services.cloudformation._


cf: AmazonCloudFormationClient = com.amazonaws.services.cloudformation.AmazonCloudFormationClient@27abdc13
defined function status
defined function statusLoop

## Creation

In [29]:
// read template
import scala.io._
import com.amazonaws.services.cloudformation.model._

val cf = new cloudformation.AmazonCloudFormationClient

// create
def createRequest() = {
    import com.amazonaws.services.cloudformation._
    import com.amazonaws.services.cloudformation.model._
    val cfr = new CreateStackRequest()
    cfr.setStackName(stackName)
    val params =  Seq(
      new Parameter().withParameterKey("KeyName").withParameterValue(keyName),
      new Parameter().withParameterKey("InstanceType").withParameterValue(instanceType),
      new Parameter().withParameterKey("ImageId").withParameterValue(imageId)    
    ) 
    cfr.setParameters(params.asJava)
    cfr
}

if(create) {
    val cfr = createRequest()
    val body = Source.fromFile(template.toString).getLines.mkString("\n")
    cfr.setTemplateBody(body)
    // run request
    val res = cf.createStack(cfr)
    statusLoop
}

import scala.io._

import com.amazonaws.services.cloudformation.model._


cf: AmazonCloudFormationClient = com.amazonaws.services.cloudformation.AmazonCloudFormationClient@18944d0a
defined function createRequest

## Cleanup

In [30]:
// remove
def removeRequest() = 
    new DeleteStackRequest().withStackName(stackName)

if(remove) { 
    cf.deleteStack(removeRequest())
    statusLoop
}
//removeRequest()

defined function removeRequest

## Building the inventory

In [30]:
import com.amazonaws.services.ec2._
import com.amazonaws.regions._

val ec2 = new AmazonEC2Client()
val regions = Regions.fromName(region)
ec2.setRegion(Region.getRegion(regions))

cmd30.sc:6: reference to Region is ambiguous;
it is imported twice in the same scope by
import com.amazonaws.regions._
and import _root_.com.amazonaws.services.ec2.model.{PermissionGroup, BundleTask, VpnState, DescribeVpcPeeringConnectionsResult, SubnetState, CreateVpnConnectionRouteRequest, ImportSnapshotTask, DescribeImportImageTasksRequest, DeleteVolumeRequest, UserBucketDetails, BundleInstanceResult, DeleteKeyPairRequest, AssignPrivateIpAddressesRequest, DescribeNetworkInterfaceAttributeRequest, DescribeFlowLogsRequest, DescribeSnapshotsRequest, S3Storage, SummaryStatus, DescribeAddressesRequest, DescribePrefixListsResult, IpPermission, Affinity, CreateTagsResult, ModifyIdentityIdFormatRequest, CreateRouteRequest, DescribeClassicLinkInstancesRequest, EbsInstanceBlockDeviceSpecification, CancelConversionTaskResult, DeleteTagsRequest, DescribeSubnetsRequest, RebootInstancesResult, CreateNetworkAclEntryRequest, ScheduledInstanceRecurrenceRequest, DetachVolumeRequest, DescribeRouteTabl

: 

In [30]:
import collection.JavaConverters._

val instances = for {
 reservation <- ec2.describeInstances.getReservations.asScala
 instance <- reservation.getInstances.asScala
} yield {
    instance
}

cmd30.sc:4: object describeInstances is not a member of package com.amazonaws.services.ec2
 reservation <- ec2.describeInstances.getReservations.asScala
                    ^

: 

In [30]:
import com.amazonaws.services.ec2.model._
import collection.JavaConverters._

val susp = if(suspend) {
    val ids = instances.filter(_.getState.getName == "running").map(_.getInstanceId)
    Some(ec2.stopInstances(new StopInstancesRequest(ids.asJava)))
    message("SUSPENDING")
} else None

val resm = if(resume) {
     val stopped = instances.filter(_.getState.getName == "stopped").map(_.getInstanceId)
     Some(ec2.startInstances(new StartInstancesRequest(stopped.asJava)))
     message("RESUMING")
} else None

cmd30.sc:6: object stopInstances is not a member of package com.amazonaws.services.ec2
    Some(ec2.stopInstances(new StopInstancesRequest(ids.asJava)))
             ^cmd30.sc:12: object startInstances is not a member of package com.amazonaws.services.ec2
     Some(ec2.startInstances(new StartInstancesRequest(stopped.asJava)))
              ^

: 